In [1]:
from genetic_algorithm import *
from numpy.random import default_rng
import matplotlib.pyplot as plt

In [2]:

# ## hexadecimal generator

nr_of_input = 3
nr_of_actions = 5
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## world size
world_size_x = 20
world_size_y = 20

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)

# ## initial brain and position generator

result = calculate_individual_output_weights(individuals)

for indiv in result:
    result[indiv]['position'] = [[x[indiv], y[indiv]]]

print(result)



{0: {'out1': 0.0, 'out4': 1.0, 'out2': 0.503, 'out3': 0.014, 'position': [[6, 10]]}, 1: {'out3': 0.004, 'out4': 0.24, 'out1': 0.0, 'out0': 0.0, 'position': [[16, 1]]}, 2: {'out0': 0.977, 'out3': 0.999, 'out2': 0.879, 'out4': 0.037, 'position': [[17, 0]]}, 3: {'out2': 0.003, 'out4': 0.042, 'out3': 0.531, 'out0': 0.936, 'out1': 0.0, 'position': [[5, 2]]}}


In [9]:
result = {0: {'out3': 1.0, 'out2': 0.087, 'out0': 0.995, 'out4': 0.992, 'out1': 0.069, 'position': [[14, 4]]},
          1: {'out1': 0.004, 'out2': 0.318, 'out4': 0.966, 'out3': 0.005, 'out0': 0.997, 'position': [[4, 5]]},
          2: {'out4': 1.0, 'out0': 0.048, 'out2': 0.88, 'out1': 0.732, 'out3': 0.831, 'position': [[3, 3]]},
          3: {'out2': 0.536, 'out1': 1.0, 'out4': 0.067, 'position': [[17, 15]]}}

In [10]:
k = 15
n = 0
while k>n:  
    for indiv in result:
        x = result[indiv]['position'][-1][0]
        y = result[indiv]['position'][-1][1]
        position_list = []
        for out in result[indiv]:
            if 'out' in out:
                new_pos = move(out, result[indiv][out])

                position_list.append(new_pos)
        
        position_list = list(map(sum, zip(*position_list)))
#         print(position_list)
        position_list = make_smaller_(position_list)
#         print(position_list)
        position_list = list(map(sum, zip(*[[x, y]] + [position_list])))
        print(position_list)
        position_list[0] = normalize_position_if_outside_world(position_list[0], world_size_x)
        position_list[1] = normalize_position_if_outside_world(position_list[1], world_size_y)
#         print('norm', position_list)
        result[indiv]['position'].append(position_list)
    n += 1
    print()        
result

[13, 4]
[3, 5]
[2, 2]
[18, 14]

[12, 4]
[2, 5]
[1, 1]
[18, 13]

[11, 4]
[1, 5]
[0, 0]
[18, 12]

[10, 4]
[1, 5]
[-1, -1]
[18, 11]

[9, 4]
[0, 5]
[0, -1]
[18, 10]

[8, 4]
[-1, 5]
[-1, -1]
[19, 9]

[7, 3]
[0, 5]
[-1, -1]
[20, 8]

[6, 3]
[0, 5]
[-1, -1]
[21, 7]

[5, 3]
[-1, 5]
[0, -1]
[20, 6]

[4, 3]
[0, 5]
[-1, -1]
[20, 5]

[3, 3]
[-1, 5]
[-1, -1]
[20, 4]

[2, 2]
[0, 5]
[-1, -1]
[20, 3]

[1, 2]
[1, 5]
[-1, -1]
[21, 2]

[0, 2]
[0, 5]
[-1, -1]
[21, 1]

[-1, 2]
[-1, 5]
[-1, -1]
[21, 0]



{0: {'out3': 1.0,
  'out2': 0.087,
  'out0': 0.995,
  'out4': 0.992,
  'out1': 0.069,
  'position': [[14, 4],
   [13, 4],
   [12, 4],
   [11, 4],
   [10, 4],
   [9, 4],
   [8, 4],
   [7, 3],
   [6, 3],
   [5, 3],
   [4, 3],
   [3, 3],
   [2, 2],
   [1, 2],
   [0, 2],
   [0, 2]]},
 1: {'out1': 0.004,
  'out2': 0.318,
  'out4': 0.966,
  'out3': 0.005,
  'out0': 0.997,
  'position': [[4, 5],
   [3, 5],
   [2, 5],
   [1, 5],
   [1, 5],
   [0, 5],
   [0, 5],
   [0, 5],
   [0, 5],
   [0, 5],
   [0, 5],
   [0, 5],
   [0, 5],
   [1, 5],
   [0, 5],
   [0, 5]]},
 2: {'out4': 1.0,
  'out0': 0.048,
  'out2': 0.88,
  'out1': 0.732,
  'out3': 0.831,
  'position': [[3, 3],
   [2, 2],
   [1, 1],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0]]},
 3: {'out2': 0.536,
  'out1': 1.0,
  'out4': 0.067,
  'position': [[17, 15],
   [18, 14],
   [18, 13],
   [18, 12],
   [18, 11],
   [18, 10],
   [19, 9],
   [20, 8],


In [1]:
l = [[13, 4], [3, 5], [2, 2], [18, 14], [18, 14], [2,2]]

In [17]:
def duplicates(l, start_index):
    for i_nr, i in enumerate(l[start_index+1:]):
#         print(i_nr, i, l[start_index])
        if l[start_index] == i:
            print(i_nr, i)
            
        duplicates(l, start_index+1)
        print()
            

In [18]:
duplicates(l, 0)

0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]







0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]







0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]







0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]







0 [18, 14]





0 [18, 14]





2 [2, 2]
0 [18, 14]






0 [18, 14]

In [22]:
[9, 8]==[9, 8]

True

In [7]:
a = [(3, 0),(4,1),(5,2)]

def slope_intercept(x1,y1,x2,y2):
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1     
    return a,b

# print(slope_intercept(3,0,2,1))

if a[-2][0] == a[-1][0]:
    x = a[-1][0]
elif a[-2][0] > a[-1][0]:
    x = a[-2][0] + 1
elif a[-2][0] < a[-1][0]:
    x = a[-2][0] - 1
# print(x)

a[-2][0]



4